In [1]:
using Eirene
using Ripserer
using CSV
using TiffImages
using Images
using NPZ
using Plots
using PersistenceDiagrams
using Measures
using Distributions
using MultivariateStats
using LinearAlgebra
using Random
using StatsBase
using JLD2
using FileIO
using PersistenceDiagrams
using DelimitedFiles
using UMAP


WebIO._IJuliaInit()

┌ Warning: Error requiring `HDF5` from `Plots`
│   exception = (LoadError("/Users/irisyoon/.julia/packages/Plots/yJrrq/src/backends/hdf5.jl", 36, UndefVarError(:Group)), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x000000010e83c2cf, Ptr{Nothing} @0x000000010e8d1206, Ptr{Nothing} @0x000000010e8d223b, Ptr{Nothing} @0x000000010e8d0b94, Ptr{Nothing} @0x000000010e8d0f0c, Base.InterpreterIP in top-level CodeInfo for Plots._hdf5_implementation at statement 4, Ptr{Nothing} @0x000000010e8eb844, Ptr{Nothing} @0x000000010e8eaf65, Ptr{Nothing} @0x000000010e8eb604, Ptr{Nothing} @0x000000010e8eb604, Ptr{Nothing} @0x000000010e8ec134, Ptr{Nothing} @0x00000001231a64e7, Ptr{Nothing} @0x000000010e8bafbf, Ptr{Nothing} @0x00000001231f7312, Ptr{Nothing} @0x0000000122e2da65, Ptr{Nothing} @0x000000010e8bafbf, Ptr{Nothing} @0x000000010e8c7839, Ptr{Nothing} @0x000000010fe164ae, Ptr{Nothing} @0x000000010e8bafbf, Ptr{Nothing} @0x000000010e8d225f, Ptr{Nothing} @0x000000010e8d0b94, Ptr{Nothing} @0x000000

In [2]:
include("src/ECM_TDA.jl")
using .ECM_TDA

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots /Users/irisyoon/.julia/packages/Plots/yJrrq/src/backends.jl:317


In [3]:
# color palettes
c1 = "#E64B35FF" # red
c2 = "#4DBBD5FF" # blue
c3 = "#00A087FF" # green
c4 = "#3C5488FF" # dark blue
c5 = "#F39B7FFF" # salmon
c6 = "#8491B4FF"

# assign colors to cell types
c_cancer = "#2EC4B6" # light blue
c_leukocytes = "#CDCDCD" # grey 
c_fibroblast= "#7209B7" # purple
c_ECM = "#EF476F" # pink
#FFC300 : yellow

celltype_colors = Dict("cancer" => c_cancer, 
                        "leukocytes"=> c_leukocytes, 
                        "fibroblast"=> c_fibroblast);

In [4]:
gr()

Plots.GRBackend()

# 1. Plot example 

In [7]:
LTX = "001"
Da = "232"
idx = 1

# load ECM image
image_path = "data/4000x4000/subregion_ECM/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".tif"
img = Array(Images.load(image_path));

# load sampled ECM
ecm_points_path = "data/4000x4000_combined/ECM_sampled/points_CSV/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".csv" 
ECM_points = CSV.read(ecm_points_path)

# load cells
cell_path = "data/4000x4000_combined/subregion_cells/LTX" * LTX * "_Da" * Da * "_idx" * string(idx) * ".csv"
cells = CSV.read(cell_path);

In [ ]:
p1 = plot(Gray.(img), frame = :box, ticks = [], size = (300, 300), background_color=:transparent, foreground_color=:black)
p2 = scatter(ECM_points[:,1], ECM_points[:,2], yflip = :true, c = c_ECM, label = "", frame = :box, ticks = [], markersize = 2, background_color=:transparent, foreground_color=:black)
plot(p1, p2, size = (600, 300))

In [ ]:
cell_types =["cancer", "leukocytes"]
plot_array = Any[]
for ct in cell_types
    # get locations of cell type
    cell_ct = cells[cells.class .== ct, :]
    c = celltype_colors[ct]
    p_ct = scatter(cell_ct.x, cell_ct.y,
                 markersize = 2,
                 yflip = true,
                 label = ct,
                 markerstrokewidth = 0.2,
                 frame = :box,
                 ticks = [],
                 c = c) 
    push!(plot_array, p_ct)
end

plot(plot_array..., layout = grid(1,2), size = (800, 400))

# 2. Compute PH for ECM

In [6]:
dir = "data/4000x4000_combined/ECM_sampled/points_CSV/"
csv_files = [item for item in walkdir(dir)][1][3:end][1];

PD0_ECM = Dict()
PD1_ECM = Dict()

for file in csv_files
    filename = split(file, ".")[1]

    df = CSV.read(dir * file)
    if size(df,1) != 0
        PD0, PD1 = run_PH(df)
    
        # save
        writedlm("data/4000x4000_combined/ECM_PD/PD0/" * filename * ".csv", PD0, ",")
        writedlm("data/4000x4000_combined/ECM_PD/PD1/" * filename * ".csv", PD1, ",")
        
        # combine to dictionary
        PD0_ECM[filename] = PD0
        PD1_ECM[filename] = PD1
    else
        PD0 = nothing
        PD1 = nothing
        
        writedlm("data/4000x4000_combined/ECM_PD/PD0/" * filename * ".csv", zeros(), ",")
        writedlm("data/4000x4000_combined/ECM_PD/PD1/" * filename * ".csv", zeros(), ",")
        # combine to dictionary
        PD0_ECM[filename] = PD0
        PD1_ECM[filename] = PD1
    end

end

In [7]:
save("data/4000x4000_combined/ECM_PD/PD.jld2", 
    "PD0_ECM", PD0_ECM,
    "PD1_ECM", PD1_ECM)

# 3. Compute persistence image

In [8]:
# convert array to Ripserer PD
PH0 = Dict(k => ECM_TDA.array_to_ripsererPD(v) for (k,v) in PD0_ECM if v != nothing)
PH1 = Dict(k => ECM_TDA.array_to_ripsererPD(v) for (k,v) in PD1_ECM if v != nothing)

# compute PI
PI0 = ECM_TDA.compute_PI(PH0);
PI1 = ECM_TDA.compute_PI(PH1);

In [23]:
# get maximum values (for plotting purposes)
PD0_ECM = Dict(k =>v for (k,v) in PD0_ECM if v != nothing)
PD1_ECM = Dict(k => v for (k,v) in PD1_ECM if v != nothing)
max0 = get_PD0_max(PD0_ECM)
max1 = get_PD1_max(PD1_ECM)

2824.7095425901757

In [24]:
# save the max values
save("data/4000x4000_combined/ECM_PD/PD_max.jld2", 
    "max0", max0,
    "max1", max1)

In [9]:
# save PI
save("data/4000x4000_combined/ECM_PD/PI.jld2", 
    "PI0", PI0,
    "PI1", PI1)

save figures of persistence diagrams and persistence images

In [33]:
### save persistence diagrams
"""
# dim 0
for k in keys(PD0_ECM)
    p = histogram(PD0_ECM[k][:,2], xlims = (0, max0), label = "")
    filename = split(idx_file[k], ".")[1]
    savefig(output_dir * "/PD0_figures/" * filename * ".png")
end

# dim 1
for k in keys(PD1_ECM)
    p = ECM_TDA.plot_PD(PD1_ECM[k], pd_min = 0, pd_max = max1, frame = :box)
    filename = split(idx_file[k], ".")[1]
    savefig(output_dir * "/PD1_figures/" * filename * ".png")
end

### save persistence images
# dim 0
for k in keys(PI0)
    filename = split(idx_file[k], ".")[1]
    p = heatmap(PI0[k])
    savefig(output_dir * "/PI0_figures/" * filename * ".png")
end

# dim 1
for k in keys(PI1)
    filename = split(idx_file[k], ".")[1]
    p = heatmap(PI1[k])
    savefig(output_dir * "/PI1_figures/" * filename * ".png")
end
"""